# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Import the data file from WeatherPy
data_file = "../output_data/weather_data.csv"
weather_data = pd.read_csv(data_file)

#Show dataframe with weather data
weather_data

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date
0,lhokseumawe,ID,81.48,71,88,2.55,5.1801,97.1507,1623203563
1,rikitea,PF,74.17,88,100,9.89,-23.1203,-134.9692,1623203563
2,hithadhoo,MV,82.69,72,10,6.60,-0.6000,73.0833,1623203563
3,huarmey,PE,66.56,83,90,7.76,-10.0681,-78.1522,1623203564
4,chokurdakh,RU,51.57,41,24,9.37,70.6333,147.9167,1623203564
...,...,...,...,...,...,...,...,...,...
577,karpogory,RU,50.13,54,46,3.29,64.0019,44.4451,1623203311
578,svetlyy,RU,53.65,89,14,6.44,54.6750,20.1347,1623203311
579,ippy,CF,69.15,96,98,0.45,6.2679,21.2247,1623203312
580,port hawkesbury,CA,70.72,60,1,2.30,45.6169,-61.3485,1623203312


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure api key for google maps
gmaps.configure(api_key=g_key)

#Determine locations using Lat and Lng, Make the weight the humidity
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [9]:
# Create the gmap figure
figure = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

#Add heat layer using lat and lng for weather data and weights as humidity
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating=False, max_intensity=100, point_radius=3)

figure.add_layer(heat_layer)

#Show map of world with heatlayer
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Narrow down cities that fit certain weather conditions(Max temp between 70 and 80, no cloudiness, Wind Speed below 10mph)
narrow_weather_data = weather_data.loc[(weather_data["Max Temp"] <= 80) & (weather_data["Max Temp"] >= 70) 
                                       & (weather_data["Wind Speed"] < 10) & (weather_data["Cloudiness"] == 0)]

#Drop any data with na
narrow_weather_data = narrow_weather_data.dropna()

#Create DataFrame with conditions met
narrow_weather_data

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date
180,sao filipe,CV,74.43,81,0,7.47,14.8961,-24.4956,1623203610
199,ginda,IN,77.88,33,0,1.36,30.6945,78.4932,1623203615
298,caravelas,BR,73.24,85,0,4.54,-17.7125,-39.2481,1623203643
323,joshimath,IN,75.96,38,0,1.41,30.5667,79.5667,1623203650
384,iquitos,PE,78.84,89,0,0.00,-3.7481,-73.2472,1623203667
438,fez,MA,70.05,60,0,1.72,34.0372,-4.9998,1623203641
502,gazanjyk,TM,72.88,47,0,2.19,39.2446,55.5154,1623203700
507,trogir,HR,71.65,49,0,2.30,43.5125,16.2517,1623203702
535,shahr-e babak,IR,77.50,13,0,4.09,30.1165,55.1186,1623203300
545,montrose,US,74.61,14,0,9.93,38.4166,-108.2673,1623203303


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Create new df with a hotel name category and display
hotel_df = narrow_weather_data
hotel_df["Hotel Name"] = ""
hotel_df 

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date,Hotel Name
180,sao filipe,CV,74.43,81,0,7.47,14.8961,-24.4956,1623203610,
199,ginda,IN,77.88,33,0,1.36,30.6945,78.4932,1623203615,
298,caravelas,BR,73.24,85,0,4.54,-17.7125,-39.2481,1623203643,
323,joshimath,IN,75.96,38,0,1.41,30.5667,79.5667,1623203650,
384,iquitos,PE,78.84,89,0,0.00,-3.7481,-73.2472,1623203667,
438,fez,MA,70.05,60,0,1.72,34.0372,-4.9998,1623203641,
502,gazanjyk,TM,72.88,47,0,2.19,39.2446,55.5154,1623203700,
507,trogir,HR,71.65,49,0,2.30,43.5125,16.2517,1623203702,
535,shahr-e babak,IR,77.50,13,0,4.09,30.1165,55.1186,1623203300,
545,montrose,US,74.61,14,0,9.93,38.4166,-108.2673,1623203303,


In [20]:
# use iterrows to go through pandas dataframe
for index, row in hotel_df.iterrows():
    try:
        
        #need base url of nearby search of wonderful condition areas with parameters of hotel within 5000 meters
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
        params = {"keyword": "hotel",
                 "radius": 5000,
                 "key": g_key}
    
        # Determine nearby search using the lat and lng of all cities in dataframe, include in params
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}"
    
        #request data using json file
        hotel_data = requests.get(base_url, params=params).json()
        
        #locate the hotel name in data and insert into dataframe
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]

    except:
        #If no hotel name put NaN
        hotel_df.loc[index, "Hotel Name"] ="NaN"

#Show dataframe with hotels
hotel_df

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date,Hotel Name
180,sao filipe,CV,74.43,81,0,7.47,14.8961,-24.4956,1623203610,Hotel Xaguate
199,ginda,IN,77.88,33,0,1.36,30.6945,78.4932,1623203615,Shri Ganga Lok Home Stay/Hotel
298,caravelas,BR,73.24,85,0,4.54,-17.7125,-39.2481,1623203643,Mar Doce de Abrolhos
323,joshimath,IN,75.96,38,0,1.41,30.5667,79.5667,1623203650,"The Cliff Top Club, Auli"
384,iquitos,PE,78.84,89,0,0.00,-3.7481,-73.2472,1623203667,DoubleTree by Hilton Iquitos
438,fez,MA,70.05,60,0,1.72,34.0372,-4.9998,1623203641,Riad Fes - Relais & Châteaux
502,gazanjyk,TM,72.88,47,0,2.19,39.2446,55.5154,1623203700,NaN
507,trogir,HR,71.65,49,0,2.30,43.5125,16.2517,1623203702,Hotel Ola
535,shahr-e babak,IR,77.50,13,0,4.09,30.1165,55.1186,1623203300,هتل هاوشک
545,montrose,US,74.61,14,0,9.93,38.4166,-108.2673,1623203303,NaN


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))